In [2]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
import gradio as gr

# Load and prepare data
df = pd.read_csv("insurance.csv")

# Label Encoding
le_sex = LabelEncoder()
le_smoker = LabelEncoder()
le_region = LabelEncoder()

df['sex'] = le_sex.fit_transform(df['sex'])
df['smoker'] = le_smoker.fit_transform(df['smoker'])
df['region'] = le_region.fit_transform(df['region'])

# Features and target
X = df.drop('charges', axis=1)
y = df['charges']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Model training
model = XGBRegressor(n_estimators=200, learning_rate=0.1, max_depth=4, random_state=42)
model.fit(X_train, y_train)

# Evaluate (optional)
y_pred = model.predict(X_test)
print("R² Score:", round(r2_score(y_test, y_pred) * 100, 2))
print("Mean Squared Error:", round(mean_squared_error(y_test, y_pred), 2))

# Gradio prediction function
def predict(age, sex, bmi, children, smoker, region):
    sex_encoded = le_sex.transform([sex])[0]
    smoker_encoded = le_smoker.transform([smoker])[0]
    region_encoded = le_region.transform([region])[0]

    input_data = np.array([[int(age), sex_encoded, float(bmi), int(children), smoker_encoded, region_encoded]])
    prediction = model.predict(input_data)[0]
    return round(prediction, 2)

# Gradio Interface
iface = gr.Interface(
    fn=predict,
    inputs=[
        gr.Dropdown(choices=[str(i) for i in range(18, 101)], label="Age"),
        gr.Dropdown(choices=["male", "female"], label="Sex"),
        gr.Dropdown(choices=[str(round(i, 1)) for i in np.arange(10.0, 60.1, 0.1)], label="BMI"),
        gr.Dropdown(choices=[str(i) for i in range(0, 6)], label="Number of Children"),
        gr.Dropdown(choices=["yes", "no"], label="Smoker"),
        gr.Dropdown(choices=["southwest", "southeast", "northwest", "northeast"], label="Region")
    ],
    outputs="number",
    title="Insurance Charges Prediction App",
    description="Select the details to estimate your insurance charges using an XGBoost regression model."
)

iface.launch()


R² Score: 87.5
Mean Squared Error: 19401682.78
It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6ef3ab8c4347aae8ed.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
